# Step 1 - Simulate data and save as obs

###IN ACCRE, Checkout the right commit of source codes (matlab and slurm scripts)

####If running a present experiment, update to the latest code:

In [ ]:
# Check out analysis code
cd /scratch/middlepg/perceptualchoice_stop_model/


git pull

# or

git fetch --all
# then
git reset --hard origin/master


####If re-running a past experiment, update to the correct commit (to be filled in by experimenter after updating):


In [ ]:
#Check out analysis code
cd /scratch/middlepg/perceptualchoice_stop_model/
git checkout 28e2aacf00973c007a2f22472fc03af53983a173 
git checkout master;


##### Step 1.1 - Add paths to MATLAB's search path

_Goal_:  
Ensure access to SAM toolbox and job code

_Instructions_:  
Paste code into MATLAB

_Output_:  
None

In [2]:
%%matlab
accreRoot = '/gpfs22';
accreHome = '/home/middlepg';
accreScratch = '/scratch/middlepg';
if isdir(fullfile(accreScratch))
    matRoot = fullfile(accreRoot,accreHome,'m-files');
    modelRoot = fullfile(accreScratch,'perceptualchoice_stop_model');
    environ = 'accre';
else
    matRoot = '/Volumes/HD-1/Users/paulmiddlebrooks/matlab';
    modelRoot = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model';
    environ = 'local';
end

addpath(genpath(fullfile(matRoot,'sam')));
addpath(genpath(fullfile(matRoot,'matlab_code_bbz')));
addpath(genpath(fullfile(matRoot,'matlab_file_exchange_tools')));
addpath(genpath(fullfile(matRoot,'cmtb')));
addpath(genpath(fullfile(modelRoot,'src/code/2016-01-08-1245')));


ERROR: Cell magic `%%matlab` not found.


##### Step 1.3 - Specify random number generator seed

_Goal_:  
Ensure reproducible findings

_Instruction_:  
Paste code into MATLAB

_Output_:  
`rngStruct.mat` in `.../data/.../preproc01/`

In [ ]:
%%matlab
preprocDataDir      = fullfile(modelRoot,'data/2016-01-08-1245/preproc01/');
rngStruct           = rng('shuffle');
save(fullfile(preprocDataDir,'rngStruct.mat'),'rngStruct');


##### Step 1.2 - Simulate data based on best fits from a specified optimization

_Goal_:  
Use best fitting parameters from a previous (or current) fit to simulate data. 

_Instructions_:  
Paste code into MATLAB

_Output_:  
`SAM_goTrials_model%.3d.mat`

In [ ]:
subject                 = 1;
model                   = 60; %[60 137 167];
architecture            = {'crace'}; %{'crace','cli','cffi'};
dt                      = 10;
trialVar                = true;
simScope                = 'go';
nSim                    = 10000;  % number of trials to simulate per factor
optimDate               = '2015-12-23-1100';
fileStr.root            = strcat(modelRoot,'/data/',optimDate,'/preproc01/subj%.2d/dt%d/%s/%s/');
fileStr.save            = strcat(modelRoot,'/data/2016-01-08-1245/preproc01/subj%.2d/dt%d/%s/%s/');
fileStr.preproc         = strcat(modelRoot,'/data/2016-01-08-1245/preproc01/');
job_simulate_data(subject,model,architecture,dt,trialVar,simScope,fileStr,nSim); 



# Step 2 - Optimize the GO parameters, using a large step size

### Step 2.1 - Build the general SAM structure

_Goal_:  
Specify general settings, including details about the experiment, model, simulation, and optimization.

_Instruction_:  
Paste into MATLAB

_Output_:  
`SAM_goTrials.mat`

In [ ]:
%%matlab
subj                    = 1;
dt                      = 10;
trialVar                = true;
simScope                = 'go';
choiceMech              = {'race'}; %{'race','ffi','li'};
stopMech                = 'none';
fileStr.rawDataDir      = fullfile(modelRoot,'data/raw/');
fileStr.preprocDataDir  = fullfile(modelRoot,'data/2016-01-08-1245/preproc01/');
load(fullfile(fileStr.preprocDataDir,'rngStruct.mat'),'rngStruct');

nChoiceMech = numel(choiceMech);

for iChoiceMech = 1:nChoiceMech
    job_spec_sam_general(subj,dt,trialVar,simScope,choiceMech{iChoiceMech},stopMech,rngStruct,fileStr);
end


### Step 2.2 - Automatic optimization of initial GO parameters

_Goal_:  
to find combinations of GO parameters with low Pearson's Chi-squared/BIC values that can be used as initial parameters for optimization. Since we're starting with optimized parameters, the initial parameters should end up being these optimzed parameters. Something would be amiss if a different set of initial parameters fit the simulated data better than the parameters from which the data were simulated.

_Instruction_:  
Run the bash script `exe_job_optim_x0.txt` on ACCRE

_Output_:  
`finalLog_initParam_goTrials_model%.3d_started%s.mat`  
`SAM_initParam_goTrials_model%.3d_started%s.mat`  

####If running on ACCRE:

#### Update code in ACCRE by committing the most recent files/changes, and pulling in perceptualchoice_stop_model folder on ACCRE.

In [ ]:
%%system
scratch
cd perceptualchoice_stop_model/src/code/2016-01-08-1245/sbatch
./exe_job_optim_x0.txt


####If running Locally:

In [ ]:
%%matlab
cd /Users/paulmiddlebrooks/perceptualchoice_stop_model/src/code/2016-01-08-1245/matlab/
exe_job_optim_x0


### Step 2.3 - Build model-specific SAM structures

_Goal_:  
to specify modeling settings that are specific to each model variant

_Instruction_:  
Paste into MATLAB  

_Output_:  
`SAM_goTrials_model%.3d.mat`

In [ ]:
subj                    = 1;
dt                      = 10;
trialVar                = true;
simScope                = 'go';
choiceMech              = {'race'}; %{'race','li','ffi'};
stopMech                = 'none';
model                   = 60;% [60 137 167];
fileStr.root            = fullfile(modelRoot,'data/2016-01-08-1245/preproc01/subj%.2d/dt%d/%s/%s/');
for iChoiceMech = 1:numel(choiceMech)
    job_spec_sam_model(subj,dt,trialVar,simScope,choiceMech{iChoiceMech},stopMech,model,fileStr);
end


### Step 2.4 - Automatic optimization of GO parameters

_Goal_:  
Find best-fitting GO parameters using constrained Simplex algorithm

_Instruction_:  
Run the bash script `exe_job_optim_x.txt` on ACCRE

_Output_:  
`finalLog_goTrials_model%.3d_startVal%.3d_started%s.mat`  
`fValAndBestX_goTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  
`SAM_goTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  

####If running on ACCRE:

In [ ]:
%%system
scratch
cd perceptualchoice_stop_model/src/code/2016-01-08-1245/sbatch
./exe_job_optim_x.txt


####If running Locally:

In [ ]:
%%matlab
cd /Users/paulmiddlebrooks/perceptualchoice_stop_model/src/code/2016-01-08-1245/matlab/
exe_job_optim_x


### Step 2.5 - Merge optimization files

_Goal_:  
Make one file containing minimized cost function value and best-fitting parameters

_Instruction_:  
Paste into MATLAB

_Output_:  
`allFValAndBestX_goTrials_model%.3d.txt`

In [ ]:
%%matlab
subj        = 1;
dt          = 10;
trialVar    = true;
optimScope  = 'go';
choiceMech  = {'race'}; %{'race', 'li', 'ffi'};
stopMech    = 'none';
model       = 60;%[60 137 167];

fileStr.root   = fullfile(modelRoot,'data/2016-01-08-1245/preproc01/subj%.2d/dt%d/%s/%s/');
fileStr.output = 'allFValAndBestX_%sTrials_model%.3d.txt';

for iChoiceMech = 1:numel(choiceMech)
    job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech,model,fileStr);
end


### Step 2.6 - Copy output to local computer

In [ ]:
%%system

scp  middlepg@login.accre.vanderbilt.edu:/scratch/middlepg/perceptualchoice_stop_model/data/2016-01-08-1245/preproc01/subj01/dt10/trialvar/crace/* /Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2016-01-08-1245/preproc01/subj01/dt10/trialvar/crace/

# STOP HERE. DID THE PARAMETER RECOVERY WORK?

===========================================================================================

# Step 3: Optimize STOP parameters

This involves fitting the stop-signal trials only. The GO parameters are fixed to the best-fitting values from Step 2.

Similar procedure as described above.

### Step 3 - Optimize the STOP parameters, using a large step size

##### Step 3.1 - Build the general SAM structure

_Goal_:  
Specify general settings, including details about the experiment, model, simulation, and optimization.

_Instruction_:  
Paste into MATLAB

_Output_:  
`SAM_stopTrials.mat`

In [ ]:
%%matlab
subj                    = 1;
dt                      = 10;
trialVar                = false;
simScope                = 'stop';
choiceMech              = {'race'};
stopMech                = 'li';
fileStr.rawDataDir      = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/raw/';
fileStr.preprocDataDir  = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/';
load(fullfile(fileStr.preprocDataDir,'rngStruct.mat'),'rngStruct');

nChoiceMech = numel(choiceMech);

for iChoiceMech = 1:nChoiceMech
    job_spec_sam_general(subj,dt,trialVar,simScope,choiceMech{iChoiceMech},stopMech,rngStruct,fileStr);
end


##### Step 3.2 - Non-automatic optimization of STOP parameters

_Goal_:  
To find good-fitting parameters by trial-and-error

_Instruction_:  
Paste code in notebook `Perceptual choice STOP x0.ipynb` into MATLAB, in GUI mode. Tweak parameters until a satisfactory fit is obtained.

_Output_:  
`userSpecX_stopTrials_model%.3d.mat`

##### Step 3.3 - Automatic optimization of initial STOP parameters

_Goal_:  
to find combinations of STOP parameters with low Pearson's Chi-squared/BIC values that can be used as initial parameters for optimization

_Instruction_:  
Run the bash script `exe_job_optim_x0.txt` on ACCRE

_Output_:  
`finalLog_initParam_stopTrials_model%.3d_started%s.mat`  
`SAM_initParam_stopTrials_model%.3d_started%s.mat`  

%%system
scratch
cd perceptualchoice_stop_model/src/code/2015-12-23-1100/sbatch
./exe_job_optim_x0.txt


##### Step 3.4 - Build model-specific SAM structures

_Goal_:  
to specify modeling settings that are specific to each model variant

_Instruction_:  
Paste into MATLAB  

_Output_:  
`SAM_stopTrials_model%.3d.mat`

In [2]:
subj                    = 1;
dt                      = 10;
trialVar                = false;
simScope                = 'stop';
choiceMech              = {'race'};
stopMech                = 'none';
model                   = [9];
fileStr.root            = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/subj%.2d/dt%d/%s/%s/';
for iChoiceMech = 1:numel(choiceMech)
    job_spec_sam_model(subj,dt,trialVar,simScope,choiceMech{iChoiceMech},stopMech,model,fileStr);
end

SyntaxError: invalid syntax (<ipython-input-2-bda021e3c66b>, line 9)

##### Step 3.5 - Automatic optimization of STOP parameters

_Goal_:  
Find best-fitting GO parameters using constrained Simplex algorithm

_Instruction_:  
Run the bash script `exe_job_optim_x.txt` on ACCRE

_Output_:  
`finalLog_stopTrials_model%.3d_startVal%.3d_started%s.mat`  
`fValAndBestX_stopTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  
`SAM_stopTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  

%%system
scratch
cd perceptualchoice_stop_model/src/code/2015-12-23-1100/sbatch
./exe_job_optim_x.txt


##### Step 3.6 - Merge optimization files

_Goal_:  
Make one file containing minimized cost function value and best-fitting parameters

_Instruction_:  
Paste into MATLAB

_Output_:  
`allFValAndBestX_stopTrials_model%.3d.txt`

# Step 4 - Optimize the GO and STOP parameters

This involves fitting the go and stop trials. The GO and STOP parameters start at the best-fitting values from Step 2 and 3.

#### Step 4.1 - Build the general SAM structure

_Goal_:  
Specify general settings, including details about the experiment, model, simulation, and optimization.

_Instruction_:  
Paste into MATLAB

_Output_:  
`SAM_allTrials.mat`

In [2]:
%%matlab
subj                    = 1; % 1:2;
dt                      = 10;
trialVar                = false;
optimScope              = 'all';
choiceMech              = 'race'; %{'race','ffi','li'};
stopMech                = 'race'; %{'race','bi','li'};
fileStr.rawDataDir      = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/raw/';
fileStr.preprocDataDir  = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/';
load(fullfile(fileStr.preprocDataDir,'rngStruct.mat'),'rngStruct');

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_spec_sam_general(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},rngStruct,fileStr);
    end
end


ERROR: Cell magic `%%matlab` not found.


##### Step 4.2 - Extract best-fitting GO and STOP parameters

In [ ]:
%%matlab
subj                    = 1; %1:2;
trialVar                = false;
optimScope              = 'all';
choiceMech              = {'race'};
stopMech                = {'race','bi','li'};
model                   = [2,5,13,10,23,33];
fileStr.root            = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/subj%.2d/dt10/%s/%s/';
fileStr.bestXSTOP       = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/subj%.2d/dt10/%s/%s/allFValAndBestX_stopTrials_model%.3d.txt';
x0Base                  = {};
doPlot                  = false;
doSave                  = true;
doStartParCluster       = false;

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);
nModel                  = numel(model);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end

choiceMech  = {'ffi'};
model       = [1 2 5 13 7 10 23 33 12 21 35];

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);
nModel                  = numel(model);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end

choiceMech  = {'li'}
model       = [1 2 5 13 6 10 23 33 11 20 34];

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);
nModel                  = numel(model);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end

##### Step 4.3 - Build model-specific SAM structure

_Goal_:  
to specify modeling settings that are specific to each model variant

_Instruction_:  
Paste into MATLAB  

_Output_:  
`SAM_allTrials_model%.3d.mat`

In [ ]:
%%matlab
subj                    = 1; %1:2; 8:13;
dt                      = 10;
trialVar                = false;
optimScope              = 'all';
choiceMech              = {'race'};
stopMech                = {'race','bi','li'};
model                   = [2,5,13,10,23,33];
fileStr.root            = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/subj%.2d/dt%d/%s/%s/';

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech              = {'ffi'};
nChoiceMech             = numel(choiceMech);
model                   = [1 2 5 13 7 10 23 33 12 21 35];
for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech              = {'li'};
nChoiceMech             = numel(choiceMech);
model                   = [1 2 5 13 6 10 23 33 11 20 34];
for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

##### Step 4.4 - Automatic optimization of GO and STOP parameters

_Goal_:  
Find best-fitting GO and STOP parameters using constrained Simplex algorithm

_Instruction_:  
Run the bash script `exe_job_optim_x.sh` on ACCRE

_Output_:  
`finalLog_allTrials_model%.3d_startVal%.3d_started%s.mat`  
`fValAndBestX_allpTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  
`SAM_allTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  

%%system
scratch
cd perceptualchoice_stop_model/src/code/2015-12-23-1100/sbatch
./exe_job_optim_x.txt


##### Step 4.5 - Merge optimization files


_Goal_:  
Make one file containing minimized cost function value and best-fitting parameters

_Instruction_:  
Paste into MATLAB

_Output_:  
`allFValAndBestX_allTrials_model%.3d.txt`

In [ ]:
%%matlab
subj        = 1; % 1:2;
dt          = 10;
trialVar    = false;
optimScope  = 'all';
choiceMech  = {'race'};
stopMech    = {'race','bi','li'};
model       = [9];

nChoiceMech = numel(choiceMech);
nStopMech   = numel(stopMech);

fileStr.root = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/subj%.2d/dt%d/%s/%s/';
fileStr.output = 'allFValAndBestX_%sTrials_model%.3d.txt';

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech  = {'ffi'};
model       = [1 2 5 13 7 10 23 33 12 21 35];

for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech  = {'li'};
model       = [1 2 5 13 6 10 23 33 11 20 34];

for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

# Step 5 - Optimize GO and STOP parameters at 1-ms resolution


This involves fitting the go and stop trials. The GO and STOP parameters start at the best-fitting values from Step 4.

##### Step 5.1 - Build the general SAM structure

_Goal_:  
Specify general settings, including details about the experiment, model, simulation, and optimization.

_Instruction_:  
Paste into MATLAB

_Output_:  
`SAM_allTrials.mat`

In [ ]:
%%matlab
subj                    = 1; %1:2;
dt                      = 1;
trialVar                = false;
optimScope              = 'all';
choiceMech              = {'race','ffi','li'};
stopMech                = {'race','bi','li'};
fileStr.rawDataDir      = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/raw/';
fileStr.preprocDataDir  = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/';
load(fullfile(fileStr.preprocDataDir,'rngStruct.mat'),'rngStruct');

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_spec_sam_general(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},rngStruct,fileStr);
    end
end

##### Step 5.2 - Extract best-fitting GO and STOP parameters at 20 ms resolution

In [ ]:
%%matlab
subj                    = 1; %1:2;
trialVar                = false;
optimScope              = 'all';
choiceMech              = {'race'};
stopMech                = {'race','bi','li'};
model                   = [2,5,13,10,23,33];
fileStr.root            = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/subj%.2d/dt1/%s/%s/';
fileStr.bestXSTOP       = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/subj%.2d/dt20/%s/%s/allFValAndBestX_allTrials_model%.3d.txt';
x0Base                  = {};
doPlot                  = false;
doSave                  = true;
doStartParCluster       = false;

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);
nModel                  = numel(model);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end

choiceMech  = {'ffi'};
model       = [1 2 5 13 7 10 23 33 12 21 35];

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);
nModel                  = numel(model);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end

choiceMech  = {'li'}
model       = [1 2 5 13 6 10 23 33 11 20 34];

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);
nModel                  = numel(model);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        for iModel = 1:nModel
            job_spec_x0base(subj,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model(iModel),fileStr,x0Base,doPlot,doSave,doStartParCluster);
        end
    end
end

##### Step 5.3 - Build model-specific SAM structure

_Goal_:  
to specify modeling settings that are specific to each model variant

_Instruction_:  
Paste into MATLAB  

_Output_:  
`SAM_allTrials_model%.3d.mat`

In [ ]:
%%matlab
subj                    = 1; %1:2; 8:13;
dt                      = 1;
trialVar                = false;
optimScope              = 'all';
choiceMech              = {'race'};
stopMech                = {'race','bi','li'};
model                   = [2,5,13,10,23,33];
fileStr.root            = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/subj%.2d/dt%d/%s/%s/';

nChoiceMech             = numel(choiceMech);
nStopMech               = numel(stopMech);

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech              = {'ffi'};
nChoiceMech             = numel(choiceMech);
model                   = [1 2 5 13 7 10 23 33 12 21 35];
for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech              = {'li'};
nChoiceMech             = numel(choiceMech);
model                   = [1 2 5 13 6 10 23 33 11 20 34];
for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_spec_sam_model(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

##### Step 5.4 - Automatic optimization of GO and STOP parameters


_Goal_:  
Find best-fitting GO and STOP parameters using constrained Simplex algorithm

_Instruction_:  
Run the bash script `exe_job_optim_x.sh` on ACCRE

_Output_:  
`finalLog_allTrials_model%.3d_startVal%.3d_started%s.mat`  
`fValAndBestX_allpTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  
`SAM_allTrials_model%.3d_startVal%.3d_exit%.1d_started%s.mat`  

%%system
scratch
cd perceptualchoice_stop_model/src/code/2015-12-23-1100/sbatch
./exe_job_optim_x.txt


In [ ]:
#!/bin/bash
# 
# exe_job_optim_x.txt 
# 
# N.B. Make sure that file's permissions allow others to execute this file  (e.g. 'chmod 755')
# N.B. Make sure that you update path strings and PBS .txt-file below

pathStr="/scratch/middlepg/perceptualchoice_stop_model/data/2015-12-17-1312/subj%.2d/dt%d/%s/%s/SAM_%sTrials_model%.3d.mat";

# Optimization scope
optimScope="all";

# Time step
dt="1";

# Trial variability tag (for starting point, non-decision time, and rate)
trialVar="notrialvar";

# Model architectures
modelArch="crace_irace crace_ibi crace_ili";

# Model variants
model="9";

# Subject
subject="1";

# PBS Settings
WALLTIME="10:00:00";
PPN="4";
MEM="12000";

for iModelArch in `echo $modelArch`
do
	for iModel in `echo $model`
	do
		for iSubj in `echo $subject`
		do
			for iStartVal in {1..1}
			do
				qsub -l walltime=`echo $WALLTIME` -l nodes=1:ppn=`echo $PPN` -l mem=`echo "$MEM"mb` -v pathStr=`echo "$pathStr"`,subject=`echo "$iSubj"`,optimScope=`echo "$optimScope"`,dt=`echo "$dt"`,trialVar=`echo "$trialVar"`,modelArch=`echo "$iModelArch"`,iModel=`echo "$iModel"`,iStartVal=`echo "$iStartVal"`,processorsPerNode=`echo "$PPN"` /scratch/zandbeb/perceptualchoice_stop_model/src/code/2015-12-17-1312/pbs/pbs_job_optim_x.txt
			done
		done
	done		
done    


##### Step 5.5 - Merge optimization files


_Goal_:  
Make one file containing minimized cost function value and best-fitting parameters

_Instruction_:  
Paste into MATLAB

_Output_:  
`allFValAndBestX_allTrials_model%.3d.txt`

In [ ]:
%%matlab
subj        = 1; % 1:2;
dt          = 1;
trialVar    = false;
optimScope  = 'all';
choiceMech  = {'race'};
stopMech    = {'race','bi','li'};
model       = [9];

nChoiceMech = numel(choiceMech);
nStopMech   = numel(stopMech);

fileStr.root = '/Volumes/HD-1/Users/paulmiddlebrooks/perceptualchoice_stop_model/data/2015-12-23-1100/preproc01/subj%.2d/dt%d/%s/%s/';
fileStr.output = 'allFValAndBestX_%sTrials_model%.3d.txt';

for iChoiceMech = 1:nChoiceMech
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech  = {'ffi'};
model       = [1 2 5 13 7 10 23 33 12 21 35];

for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end

choiceMech  = {'li'};
model       = [1 2 5 13 6 10 23 33 11 20 34];

for iChoiceMech = 1:numel(choiceMech)
    for iStopMech = 1:nStopMech
        job_merge_fvalandbestx(subj,dt,trialVar,optimScope,choiceMech{iChoiceMech},stopMech{iStopMech},model,fileStr);
    end
end